<a href="https://colab.research.google.com/github/SamMeacham/cardcutting/blob/main/Search_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai


# Card Cutting

In [ ]:
import os

from openai import OpenAI
os.environ['OPENAI_API_KEY'] = ###KEY##

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an assistant specialized in summarizing and highlighting key information from extensive texts. Focus on extracting the most important lines based on the topics specified by the user."},
    {"role": "user", "content": "Identify the most important lines from the following article, emphasizing the topics [insert topics here]. Here's the article: [insert article content here]."}
  ]
)

print(completion.choices[0].message)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-81ijP***************************************Lpab. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
article = "But mostly diseases don't drive species extinct. There are several reasons for that. For one, the most dangerous diseases are those that spread from one individual to another. If the disease is highly lethal, then the population drops, and it becomes less likely that individuals will contact each other during the infectious phase. Highly contagious diseases tend to burn themselves out that way.¶ Probably the main reason is variation. Within the host and the pathogen population there will be a wide range of variants. Some hosts may be naturally resistant. Some pathogens will be less virulent. And either alone or in combination, you end up with infected individuals who survive.¶ We see this in HIV, for example. There is a small fraction of humans who are naturally resistant or altogether immune to HIV, either because of their CCR5 allele or their MHC Class I type. And there are a handful of people who were infected with defective versions of HIV that didn't progress to disease. ¶ We can see indications of this sort of thing happening in the past, because our genomes contain many instances of pathogen resistance genes that have spread through the whole population. Those all started off as rare mutations that conferred a strong selection advantage to the carriers, meaning that the specific infectious diseases were serious threats to the species."

In [ ]:
#Underlining
def underline(client, article_text, topics):
    """
    Calls the GPT API to extract the most important lines from an article,
    focusing on specified topics.

    Parameters:
    - client: The instance of the OpenAI API client.
    - article_text: The full text of the article to analyze.
    - topics: A list of topics or arguments the user is interested in.

    Returns:
    A string containing the summarized important lines as extracted by the API.
    """
    # Ensure topics is a list for proper processing
    if isinstance(topics, str):
        topics = [topics]

    # Convert the list of topics into a comma-separated string
    topics_str = ", ".join(topics)

    # Construct the API call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specialized in extracting key lines from news articles and policy briefs. You are focused on brevity and should only give me back the most important sentences based on the arguments specified by the user. Do not modify or edit the text in any way. You should only choose the most important lines as they are represented in the text"},
            {"role": "user", "content": f"I want lines that prove and support the following arguments: {topics_str}. Here's the article: {article_text}"}
        ]
    )

    # Correct way to access the content of the first choice's message
    highlighted_lines = response.choices[0].message.content if response.choices else ""

    return highlighted_lines


# New Section

In [ ]:
#highlight
def highlight(client, underlined, topics):
    """
    Takes important lines extracted from an article and distills them into
    the most critical phrases or words to construct an argument.

    Parameters:
    - client: The instance of the OpenAI API client.
    - important_lines: A string containing the important lines as extracted by the previous API call.
    - topics: A list of topics or arguments to guide the extraction of key phrases.

    Returns:
    A string with the most critical phrases or words highlighted.
    """
    # Ensure topics is a list for proper processing
    if isinstance(topics, str):
        topics = [topics]

    # Convert the list of topics into a comma-separated string
    topics_str = ", ".join(topics)

    # Construct the API call with a new prompt
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specialized in identifying and highlighting key phrases within specific sentences. Your goal is to extract and highlight the phrases that are most crucial for constructing an argument based on specified topics. Highlight only the essential phrases."},
            {"role": "user", "content": f"From the following important lines, extract and highlight the phrases most crucial for supporting the arguments: {topics_str}. Here are the important lines: {underlined}"}
        ]
    )

    # Extracting the highlighted phrases from the response
    highlighted_phrases = response.choices[0].message.content if response.choices else ""

    return highlighted_phrases



In [ ]:
underlined = underline(client, article, "I want lines that prove that diseases will not cause extinction because they kill their hosts too quickly")

In [ ]:
highlighted = highlight(client, underlined,  "I want lines that prove that diseases will not cause extinction because they kill their hosts too quickly")

In [ ]:
print(underlined)
print(highlighted)

ModuleNotFoundError: No module named 'docx'

In [ ]:
from docx import Document
from docx.shared import RGBColor, Pt
from docx.enum.text import WD_COLOR_INDEX, WD_UNDERLINE

def format_article_with_highlights(document_text, important_lines, highlighted_phrases):
    """
    Creates a Word document from the original article text, with specific
    lines underlined and specific phrases highlighted.

    Parameters:
    - document_text: The full text of the original article.
    - important_lines: A list of important lines to be underlined.
    - highlighted_phrases: A list of phrases to be highlighted.

    Saves the document as 'formatted_article.docx'.
    """
    doc = Document()
    para = doc.add_paragraph()

    # Split the document into lines for easier processing
    lines = document_text.split('\n')

    for line in lines:
        added = False
        # Check if the line is one of the important lines
        if line in important_lines:
            run = para.add_run(line + '\n')
            run.underline = True
            added = True

        # Check for and highlight phrases within the line
        for phrase in highlighted_phrases:
            if phrase in line:
                # If the line hasn't been added already
                if not added:
                    start_index = line.find(phrase)
                    before_phrase = line[:start_index]
                    # Add text before the phrase without formatting
                    if before_phrase:
                        para.add_run(before_phrase)
                    # Add the phrase with highlighting
                    highlighted_run = para.add_run(phrase)
                    highlighted_run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                    # Add remaining part of the line
                    after_phrase = line[start_index + len(phrase):]
                    if after_phrase:
                        para.add_run(after_phrase + '\n')
                else:
                    # If the line was already added because it's important, highlight the phrase
                    highlighted_run = para.add_run(phrase)
                    highlighted_run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                added = True
                break  # Assuming one phrase per line for simplicity

        # If the line was neither important nor contained a highlighted phrase, add it as is
        if not added:
            para.add_run(line + '\n')

    doc.save('formatted_article.docx')
    print("Document 'formatted_article.docx' has been created and saved.")


ModuleNotFoundError: No module named 'docx'

# Search

In [ ]:
!pip install exa_py
from exa_py import Exa
exa = Exa("447f6591-0c8c-4a5e-8c52-8d35a5cd5d1f")


In [ ]:
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = #####
client = OpenAI()

In [ ]:
def translatetometaphor(task):
    """
    Calls GPT to transform a detailed task description into a one-sentence summary.

    Parameters:
    - task: A string describing the user's specific interest or query.

    Returns:
    A string containing a one-sentence description of the desired article.
    """

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specialized in shortening long descriptions of desired articles into a one-sentence description of the desired article. Focus on retaining the key components of the desired article. Please keep the sentence relatively short, around at least less than 15 or 20 words"},
            {"role": "user", "content": f"Please transform the following search term into a one-sentence description of the desired article: {task}"}
        ]
    )

    shortened_task = completion.choices[0].message.content
    return shortened_task


In [ ]:
task = "I want an article that argues that diseases will not cause extinction. The article should argue that disease will not kill everyone."
print(translatetometaphor(task))


Article arguing diseases won't cause human extinction but won't kill everyone.


In [ ]:
def find_most_relevant_article(texts, task):
    """
    Calls GPT to compare the texts of ten articles against a user-defined task and identify the most relevant article.

    Parameters:
    - client: The instance of the OpenAI API client.
    - texts: A list of strings, where each string is the text of an article.
    - task: A string describing the user's specific interest or query.

    Returns:
    An integer index of the article in the `texts` list that is most relevant to the task.
    """

    # Combine all texts into a single string, each preceded by its index for identification
    combined_texts = "\n\n".join([f"Article {i+1}: {text}" for i, text in enumerate(texts[:5])])

    # Construct the API call with a custom prompt
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant specialized in analyzing texts to determine which one most closely matches a specified topic or task. Identify the most relevant text based on the task description."},
            {"role": "user", "content": f"Task: {task}\n\n{combined_texts}\n\nWhich article is most relevant to the above task? In your output, provide solely the number of the article you chose. Do not provide anything else besides the number. Do not say 'Article' before the number. Only put the number, so there should be no letters in your response."}
        ]
    )

    # Assuming the response is in the expected format, parse it to find the most relevant article
    if response.choices and response.choices[0].message.content:
      return response.choices[0].message.content
    else:
        return "Unable to determine the most relevant article due to an unexpected response format."

# Usage example:
# Assuming `client` is your OpenAI client instance and you have the `texts` list and `task` string defined
# relevant_article = find_most_relevant_article(client, texts, task)
# print(relevant_article)


In [ ]:
task = "I want an article that argues the tik tok ban will not be effective"


TypeError: find_most_relevant_article() takes 2 positional arguments but 3 were given

In [ ]:
def get_article_info(gpt_index, titles, urls, authors, texts, dates):
    """
    Constructs a dictionary with the information of the chosen article based on the provided GPT output index,
    automatically adjusting for zero-based indexing.

    Parameters:
    - gpt_index: The one-based index of the chosen article as provided by GPT.
    - titles: A list of article titles.
    - urls: A list of article URLs.
    - authors: A list of article authors.
    - texts: A list of article texts.
    - dates: A list of article publication dates.

    Returns:
    A dictionary containing the title, URL, author, text, and publication date of the chosen article, after adjusting for zero-based indexing.
    """

    # Adjust for zero-based indexing
    index = int(gpt_index) - 1

    # Ensure the index is within the bounds of the lists
    if 0 <= index < len(titles):
        article_info = {
            "title": titles[index],
            "url": urls[index],
            "author": authors[index],
            "text": texts[index],
            "date": dates[index]
        }
        return article_info
    else:
        return {"error": "Index out of range. Please provide a valid article number."}

# Example usage:
# Assuming the lists `titles`, `urls`, `authors`, `texts`, and `dates` are defined, and `gpt_index` is the index from GPT output
# gpt_index = int(get_gpt_response(client, texts, task))  # Ensure you convert GPT's response to an integer
# article_info = get_article_info(gpt_index, titles, urls, authors, texts, dates)
# print(article_info)


In [ ]:
task = "I want an article that argues disease won't cause extinction. The article provides arguments that disease will not kill all humans and gives specific reasons why disease will not kill everyone"
def finalsearch(task):
  shortenedtask = translatetometaphor(task)

  search_response = exa.search_and_contents(shortenedtask, use_autoprompt=True)

  titles = [result.title for result in search_response.results]
  urls = [result.url for result in search_response.results]
  authors = [result.author for result in search_response.results]
  texts = [result.text for result in search_response.results]
  date = [result.published_date for result in search_response.results]

  gpt_index = find_most_relevant_article(texts, task)
  print(gpt_index)

  dictionary = get_article_info(gpt_index, titles, urls, authors, texts, date)

  return(dictionary)



In [241]:
task = "Here is an article arguing that the tik tok ban is ineffective"
test = finalsearch(task)


2


In [242]:
print(test)

{'title': 'Opinion | Banning TikTok is a bad solution to the wrong problem', 'url': 'https://www.washingtonpost.com/opinions/2023/03/15/tiktok-ban-propaganda-bytedance-china/', 'author': 'Chris Stokel-Walker', 'text': '    Chris Stokel-Walker is a British journalist who writes about technology. He is the author of “TikTok Boom: China’s Dynamite App and the Superpower Race for Social Media.”  No more glow-ups. No more 10-second viral dances. No more short lessons on how to clean your stove. Banning TikTok would potentially save users a lot of time scrolling through incredibly addictive content.  But a ban would be an entirely un-American, undemocratic and inappropriate response to an unproven risk that the Chinese-owned platform will share users’ data with Beijing for nefarious purposes. What’s more, banning TikTok would be completely useless in combating a different, much better-evidenced social media pitfall — the spread of dangerous propaganda. An outright ban of the social media app

In [ ]:
print(gpt_index)

3


In [ ]:
dictionary = get_article_info(gpt_index, titles, urls, authors, texts, dates)

NameError: name 'dates' is not defined